검색 키워드 기준 수집 크롤러

2024.08.02 기준 수정 ver

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time
import random
from tqdm import tqdm

# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from io import BytesIO
from openpyxl.drawing.image import Image
from upload_data import *

#-----------chrome setting------------
chrome_driver_path = r'C:/Users/tips_workstation2/Desktop/new_crawler/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

query_list = ['마스크팩'] # 무선청소기, 마스크팩, 레깅스
dataframes = {}

for query in query_list:
    # list initialize
    product_names = []
    product_urls = []
    product_prices = []
    product_sellers = []
    product_details = []
    product_marketings = []
    
    for index in tqdm(range(1, 4)):
        url = f'https://search.shopping.naver.com/search/all?adQuery={query}&origQuery={query}&pagingIndex={index}&pagingSize=40&productSet=checkout&query={query}&sort=rel&timestamp=&viewType=list'
        driver.get(url)
        scroll_location = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)
            scroll_height = driver.execute_script("return document.body.scrollHeight")
            if scroll_location == scroll_height:
                break
            else:
                scroll_location = driver.execute_script("return document.body.scrollHeight")
        # Parse HTML
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        time.sleep(1)
        # Extract product elements
        products = soup.findAll('div', {'class': 'product_inner__gr8QR'})
        
        for product in products:
            try:
                # Product name
                name = product.find('div', {'class': 'product_title__Mmw2K'}).get_text()
                # Product URL
                url = product.find('a', {'class': 'product_link__TrAac linkAnchor _nlog_click _nlog_impression_element'})['href']
                # Price
                price = product.find('span', {'class': 'price_num__S2p_v'}).get_text()
                # Seller information
                # seller = product.find('a', {'class': 'product_mall__hPiEH linkAnchor'}).get_text()
                seller_box = product.find('a', {'class' : 'product_mall__hPiEH linkAnchor _nlog_click _nlog_impression_element'})
                if seller_box.find('img'):
                    seller = seller_box.find('img')['alt']
                else:
                    seller = seller_box.get_text()
                # Detail information
                detail_box = product.find('div', {'class': 'product_detail_box__QjZP8'})
                if detail_box:
                    detail_texts = [a_tag.get_text().strip() for a_tag in detail_box.find_all('a')]
                    detail = ', '.join(detail_texts)
                else:
                    detail = ''
                # Marketing phrases
                marketing_element = product.find('div', {'class': 'product_product_event__LemcX'})
                marketing = marketing_element.get_text() if marketing_element else ''
                product_names.append(name)
                product_urls.append(url)
                product_prices.append(price)
                product_sellers.append(seller)
                product_details.append(detail)
                product_marketings.append(marketing)
            except Exception as e:
                print(f"Error occurred: {e}")
    
    
    dataframes[query] = pd.DataFrame({'product_names' : product_names,
                            'product_urls' : product_urls,
                            'product_prices' : product_prices,
                            'prodict_sellers' : product_sellers,
                            'product_detail' : product_details,
                            'product_marketings' : product_marketings
                            })
    # 카테고리마다 변경
    ecommerce = 'naver'
    crawling_type = 'meta'
    category = 'Beauty'
    search_type = 'keyword'
    save_meta_query_data(dataframes[query], ecommerce, crawling_type, category, search_type, query)
driver.quit()

ctg4 id 기준

In [9]:
import pandas as pd

meta_categoty = pd.read_excel("C:/Users/tips_workstation2/Downloads/meta_category.xlsx")


In [10]:
meta_categoty

,naver_category4_id,revised_naver_category4_name,ailee_category1_id,ailee_category_1_name
0,50005468,유아음료,17,출산/육아용품
1,50005467,유아유제품,17,출산/육아용품
2,50000738,유아두유,17,출산/육아용품
3,50000739,유아과자,17,출산/육아용품
4,50000617,해열시트,17,출산/육아용품
...,...,...,...,...
2406,50006739,사료통/사료스푼,19,반려동물용품
2407,50006736,자동급식기,19,반려동물용품
2408,50006735,식기/식탁,19,반려동물용품
2409,50006737,급수기/물병,19,반려동물용품


In [11]:
meta_categoty.dropna(axis = 0, inplace = True)

In [19]:
meta_categoty[meta_categoty['ailee_category1_id'] == 17][15:]

,naver_category4_id,revised_naver_category4_name,ailee_category1_id,ailee_category_1_name
15,50000611,분통/파우더통,17,출산/육아용품
16,50000610,기저귀휴지통,17,출산/육아용품
18,50000532,우주복,17,출산/육아용품
19,50000531,신생아모자/보닛,17,출산/육아용품
20,50000530,손/발싸개,17,출산/육아용품
...,...,...,...,...
397,50000271,상하세트,17,출산/육아용품
398,50007121,댄스복,17,출산/육아용품
399,50007125,레인코트,17,출산/육아용품
400,50007116,레깅스,17,출산/육아용품


In [26]:
meta_lst = meta_categoty[meta_categoty['ailee_category1_id'] == 17][15:50]['naver_category4_id'].to_list()
len(meta_lst)

35

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time
import random
from tqdm import tqdm

# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from io import BytesIO
from openpyxl.drawing.image import Image
from upload_data import *

#-----------chrome setting------------
chrome_driver_path = r'C:/Users/tips_workstation2/Desktop/new_crawler/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

ctg4_id = ['50005468' ,'50005467', '50000738', '50000739', '50000617']

dataframes = {}

for id in ctg4_id:
    # list initialize
    product_names = []
    product_urls = []
    product_prices = []
    product_sellers = []
    product_details = []
    product_marketings = []
    
    for index in tqdm(range(1, 4)):
        # url = f'https://search.shopping.naver.com/search/category/?adQuery&catId={ctg4_id}&iq=%EC%9A%95%EC%8B%A4%EB%82%9C%EB%B0%A9%EA%B8%B0&origQuery=&pagingIndex={index}&pagingSize=40&productSet=checkout'
        url = f'https://search.shopping.naver.com/search/category?adQuery=&catId={id}&origQuery=&pagingIndex={index}&pagingSize=40&productSet=checkout&query=&sort=rel&timestamp=&viewType=list'
        driver.get(url)
        scroll_location = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)
            scroll_height = driver.execute_script("return document.body.scrollHeight")
            if scroll_location == scroll_height:
                break
            else:
                scroll_location = driver.execute_script("return document.body.scrollHeight")
        # Parse HTML
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        time.sleep(1)
        # Extract product elements
        products = soup.findAll('div', {'class': 'product_inner__gr8QR'})
        
        for product in products:
            try:
                # 상품 이름
                name_box = product.find('div', {'class': 'product_title__Mmw2K'})
                name = name_box.get_text() if name_box else 'N/A'

                # 상품 URL
                url_box = product.find('a', {'class': 'product_link__TrAac linkAnchor'})
                url = url_box['href'] if url_box else 'N/A'

                # 가격
                price_box = product.find('span', {'class': 'price_num__S2p_v'})
                price = price_box.get_text() if price_box else 'N/A'

                # 판매자 정보
                seller_box = product.find('a', {'class' : 'product_mall__hPiEH linkAnchor'})
                if seller_box:
                    if seller_box.find('img'):
                        seller = seller_box.find('img')['alt']
                    else:
                        seller = seller_box.get_text()
                else:
                    seller = 'N/A'

                # 상세 정보
                detail_box = product.find('div', {'class': 'product_detail_box__QjZP8'})
                if detail_box:
                    detail_texts = [a_tag.get_text().strip() for a_tag in detail_box.find_all('a')]
                    detail = ', '.join(detail_texts)
                else:
                    detail = ''

                # 마케팅 문구
                marketing_element = product.find('div', {'class': 'product_product_event__LemcX'})
                marketing = marketing_element.get_text() if marketing_element else ''

                product_names.append(name)
                product_urls.append(url)
                product_prices.append(price)
                product_sellers.append(seller)
                product_details.append(detail)
                product_marketings.append(marketing)

            except Exception as e:
                print(f"Error occurred: {e}")
    
    dataframes[id] = pd.DataFrame({'product_names' : product_names,
                            'product_urls' : product_urls,
                            'product_prices' : product_prices,
                            'prodict_sellers' : product_sellers,
                            'product_detail' : product_details,
                            'product_marketings' : product_marketings
                            })
    # 카테고리마다 변경
    ecommerce = 'naver'
    crawling_type = 'meta'
    category = '출산-유아용품'
    search_type = 'ctg4'
    save_meta_ctg4_data(dataframes[id], ecommerce, crawling_type, category, search_type, id)

driver.quit()